# Import

In [13]:
import boto3
import sys
import json
print('Running boto3 version:', boto3.__version__)

Running boto3 version: 1.35.32


# Testing connection

In [2]:
import boto3
from botocore.exceptions import NoCredentialsError, PartialCredentialsError

def test_aws_connection():
    try:
        # Create an S3 client (or use another AWS service)
        s3 = boto3.client('s3')
        
        # Attempt to list S3 buckets
        response = s3.list_buckets()
        
        # Print out bucket names to confirm connection
        print("Connected successfully! Buckets:")
        for bucket in response['Buckets']:
            print(f"  {bucket['Name']}")
    except NoCredentialsError:
        print("AWS credentials not found.")
    except PartialCredentialsError:
        print("Incomplete AWS credentials found.")
    except Exception as e:
        print(f"Error: {str(e)}")

test_aws_connection()


Connected successfully! Buckets:
  dev-managinglife-aws-bedrock
  dev-managinglife-db-bkp
  logs-bucket-managinglife
  managemypain-archives
  managinglife-business-data
  managinglife-business-data-humana
  managinglife-config-s3
  managinglife-documents
  managinglife-drupal-db-prod
  managinglife-es-bkp
  managinglife-guardduty-s3
  managinglife-images
  managinglife-solr-bkp
  managinglife-userdata-ca
  managinglife-userdata-ca-dev
  managinglife-userdata-ca-preprod
  managinglifebucket1
  s3-logs-bucket-managinglife
  shared-managinglife-developer-s3
  terraform-managinglife


# Converse API

In [5]:
region = 'ca-central-1'

bedrock = boto3.client(
    service_name = 'bedrock-runtime',
    region_name = region,
    )

MODEL_IDS = [
    "anthropic.claude-3-sonnet-20240229-v1:0",
    ]

In [6]:
def invoke_bedrock_model(client, id, prompt, max_tokens=2000, temperature=0, top_p=0.9):
    response = ""
    try:
        response = client.converse(
            modelId=id,
            messages=[
                {
                    "role": "user",
                    "content": [
                        {
                            "text": prompt
                        }
                    ]
                }
            ],
            inferenceConfig={
                "temperature": temperature,
                "maxTokens": max_tokens,
                "topP": top_p
            }
            #additionalModelRequestFields={
            #}
        )
    except Exception as e:
        print(e)
        result = "Model invocation error"
    try:
        result = response['output']['message']['content'][0]['text'] \
        + '\n--- Latency: ' + str(response['metrics']['latencyMs']) \
        + 'ms - Input tokens:' + str(response['usage']['inputTokens']) \
        + ' - Output tokens:' + str(response['usage']['outputTokens']) + ' ---\n'
        return result
    except Exception as e:
        print(e)
        result = "Output parsing error"
    return result

In [7]:
prompt = ("What is the capital of Italy?")
print(f'Prompt: {prompt}\n')

for i in MODEL_IDS:
    response = invoke_bedrock_model(bedrock, i, prompt)
    print(f'Model: {i}\n{response}')

Prompt: What is the capital of Italy?

Model: anthropic.claude-3-sonnet-20240229-v1:0
The capital of Italy is Rome.
--- Latency: 325ms - Input tokens:14 - Output tokens:10 ---



# Test completion API

In [10]:
def generate_message(bedrock_runtime, model_id, messages, max_tokens=512,top_p=1,temp=0.5,system=''):

    body=json.dumps(
        {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": max_tokens,
            "messages": messages,
            "temperature": temp,
            "top_p": top_p,
            "system": system
        }  
    )  
    
    response = bedrock_runtime.invoke_model(body=body, modelId=model_id)
    response_body = json.loads(response.get('body').read())

    return response_body

In [11]:
bedrock_client = boto3.client('bedrock-runtime',region_name='ca-central-1')

In [14]:
messages=[{ "role":'user', "content":[{'type':'text','text': "What is quantum mechanics? "}]},\
         { "role":'assistant', "content":[{'type':'text','text': "It is a branch of physics that \
         describes how matter and energy interact with discrete energy values "}]},\
         { "role":'user', "content":[{'type':'text','text': "Can you explain a bit more about discrete energies?"}]}]

generate_message(bedrock_client, model_id = 'anthropic.claude-3-sonnet-20240229-v1:0',messages=messages,max_tokens=512,temp=0.5,top_p=0.9)

{'id': 'msg_bdrk_01GruxojppjRg8UArw1pDQ2f',
 'type': 'message',
 'role': 'assistant',
 'model': 'claude-3-sonnet-20240229',
 'content': [{'type': 'text',
   'text': 'Sure, the concept of discrete or quantized energies is a key principle of quantum mechanics. It states that the energy of particles, atoms, and molecules cannot take on any arbitrary value, but is instead restricted to specific discrete values or quanta.\n\nSome key points about discrete energies:\n\n- Classical physics assumed energy could vary continuously. Quantum mechanics showed that energy is quantized into discrete units called quanta.\n\n- The allowed energy levels depend on the specific system (atom, molecule, particle, etc.). Each system has its own set of permissible discrete energy levels.\n\n- Particles/atoms/molecules can only exist in these specific energy states - they cannot have energies between the allowed levels.\n\n- When a system absorbs or emits energy, it must do so in discrete quantized amounts equ

## Messages with System (Personas)

In [16]:
messages = [{ "role":'user', "content":[{'type':'text','text': "What is quantum mechanics?"}]}]

system = "Respond in a way a caveman would understand"
generate_message(bedrock_client, model_id = "anthropic.claude-3-sonnet-20240229-v1:0",messages=messages,max_tokens=512,temp=0.5,top_p=0.9,system=system)

{'id': 'msg_bdrk_01H5FgrMhA6Q2oFcfBjcoYD4',
 'type': 'message',
 'role': 'assistant',
 'model': 'claude-3-sonnet-20240229',
 'content': [{'type': 'text',
   'text': "Here's how I would explain quantum mechanics in a way a caveman could understand:\n\nQuantum mechanics strange rules that tiny things follow. Things so small, like little rocks that make up everything. These tiny rocks act very weird, not like normal big rocks we see.\n\nNormal rocks follow simple rules we understand. If you throw rock, it goes where you throw it. But tiny quantum rocks don't follow those rules. They can be in many places at once, like magic! Or they can tunnel through barriers they shouldn't be able to.\n\nQuantum rocks also get connected in spooky ways. If you have two quantum rocks and you make them touch, they get linked. Even if you separate them very far apart, what happens to one instantly affects the other, like voodoo magic!\n\nThe rules of quantum mechanics are very bizarre and unlike anything we